## 전체 사용자에 대해서 step, sleep, heart 데이터를 1일 평균으로 구함

In [2]:
import pandas as pd
import json 
import os
import csv

In [3]:
def readFile(filename):
    f = open(filename, 'r')
    js = json.loads(f.read())
    f.close()
    return js

In [19]:
steps = []
day = []
sleep = []
hearts = []

In [20]:
dates = pd.date_range("20160401","20160520")
for date in dates :
    personal_steps = 0
    personal_sleep = 0
    personal_hearts = 0
    for person_number in range(1,98): #실제는 68명
        try:
            str_step = "sokulee/A0"+str(person_number)+"/A0"+str(person_number)+"_20"+str(date.strftime("%y%m%d"))+"_steps.json"
            str_sleep = "sokulee/A0"+str(person_number)+"/A0"+str(person_number)+"_20"+str(date.strftime("%y%m%d"))+"_sleep.json"
            str_hearts = "sokulee/A0"+str(person_number)+"/A0"+str(person_number)+"_20"+str(date.strftime("%y%m%d"))+"_heart.json"
            day_steps = readFile(str_step)  
            day_sleep = readFile(str_sleep)  
            day_hearts = readFile(str_hearts)
            try:
                personal_steps = personal_steps + int(day_steps['activities-steps'][0]['value'])
                personal_sleep = personal_sleep + int(day_sleep['summary']['totalMinutesAsleep'])
                personal_hearts = personal_hearts + int(day_hearts["activities-heart"][0]['value']['heartRateZones'][1]['minutes'])
            except KeyError: 
                continue
        except FileNotFoundError:
            continue
    str_tmp = "20"+str(date.strftime("%y%m%d"))
    day.append(str_tmp)
    steps.append(personal_steps/68)
    sleep.append(personal_sleep/68)
    hearts.append(personal_hearts/68)

In [21]:
total_day = pd.DataFrame({'Date' : day ,'step_count' : steps, 'sleep_time' : sleep, 'wear_time' : hearts})
total_day

,Date,sleep_time,step_count,wear_time
0,20160401,336.382353,11254.750000,131.058824
1,20160402,319.073529,9983.808824,100.588235
2,20160403,218.132353,4636.411765,45.250000
3,20160404,321.191176,15981.073529,149.573529
4,20160405,321.235294,13701.308824,149.279412
5,20160406,331.176471,12669.882353,113.705882
6,20160407,323.235294,13513.985294,113.308824
7,20160408,366.073529,15802.529412,159.764706
8,20160409,361.750000,13777.676471,141.102941
9,20160410,405.029412,8412.647059,99.676471


In [22]:
filename = 'total_day.csv'
total_day.to_csv(filename, index=False, encoding='utf-8')

# 일, 시간 heatmap

In [11]:
total_step = [0] * 168

In [12]:
dates = pd.date_range("20160401","20160520")
for date in dates :
    for person_number in range(1,98): #실제는 68명
        try:
            str_step = "sokulee/A0"+str(person_number)+"/A0"+str(person_number)+"_20"+str(date.strftime("%y%m%d"))+"_steps.json"
            print(str_step)
            day_steps = readFile(str_step)  
            try:
                now_date = day_steps['activities-steps'][0]['dateTime']

                now_weekday = pd.to_datetime(now_date).weekday()
                
                intraday = day_steps['activities-steps-intraday']['dataset']
                for minute in intraday:
                    if int(minute['value']) != 0:
                        total_step[(now_weekday * 24) + int(minute['time'][:2])] += int(minute['value'])
            except KeyError: 
                continue
        except FileNotFoundError:
            continue
    str_tmp = "20"+str(date.strftime("%y%m%d"))

sokulee/A01/A01_20160401_steps.json
sokulee/A02/A02_20160401_steps.json
sokulee/A03/A03_20160401_steps.json
sokulee/A04/A04_20160401_steps.json
sokulee/A05/A05_20160401_steps.json
sokulee/A06/A06_20160401_steps.json
sokulee/A07/A07_20160401_steps.json
sokulee/A08/A08_20160401_steps.json
sokulee/A09/A09_20160401_steps.json
sokulee/A010/A010_20160401_steps.json
sokulee/A011/A011_20160401_steps.json
sokulee/A012/A012_20160401_steps.json
sokulee/A013/A013_20160401_steps.json
sokulee/A014/A014_20160401_steps.json
sokulee/A015/A015_20160401_steps.json
sokulee/A016/A016_20160401_steps.json
sokulee/A017/A017_20160401_steps.json
sokulee/A018/A018_20160401_steps.json
sokulee/A019/A019_20160401_steps.json
sokulee/A020/A020_20160401_steps.json
sokulee/A021/A021_20160401_steps.json
sokulee/A022/A022_20160401_steps.json
sokulee/A023/A023_20160401_steps.json
sokulee/A024/A024_20160401_steps.json
sokulee/A025/A025_20160401_steps.json
sokulee/A026/A026_20160401_steps.json
sokulee/A027/A027_20160401_ste

sokulee/A077/A077_20160403_steps.json
sokulee/A078/A078_20160403_steps.json
sokulee/A079/A079_20160403_steps.json
sokulee/A080/A080_20160403_steps.json
sokulee/A081/A081_20160403_steps.json
sokulee/A082/A082_20160403_steps.json
sokulee/A083/A083_20160403_steps.json
sokulee/A084/A084_20160403_steps.json
sokulee/A085/A085_20160403_steps.json
sokulee/A086/A086_20160403_steps.json
sokulee/A087/A087_20160403_steps.json
sokulee/A088/A088_20160403_steps.json
sokulee/A089/A089_20160403_steps.json
sokulee/A090/A090_20160403_steps.json
sokulee/A091/A091_20160403_steps.json
sokulee/A092/A092_20160403_steps.json
sokulee/A093/A093_20160403_steps.json
sokulee/A094/A094_20160403_steps.json
sokulee/A095/A095_20160403_steps.json
sokulee/A096/A096_20160403_steps.json
sokulee/A097/A097_20160403_steps.json
sokulee/A01/A01_20160404_steps.json
sokulee/A02/A02_20160404_steps.json
sokulee/A03/A03_20160404_steps.json
sokulee/A04/A04_20160404_steps.json
sokulee/A05/A05_20160404_steps.json
sokulee/A06/A06_201604

sokulee/A038/A038_20160406_steps.json
sokulee/A039/A039_20160406_steps.json
sokulee/A040/A040_20160406_steps.json
sokulee/A041/A041_20160406_steps.json
sokulee/A042/A042_20160406_steps.json
sokulee/A043/A043_20160406_steps.json
sokulee/A044/A044_20160406_steps.json
sokulee/A045/A045_20160406_steps.json
sokulee/A046/A046_20160406_steps.json
sokulee/A047/A047_20160406_steps.json
sokulee/A048/A048_20160406_steps.json
sokulee/A049/A049_20160406_steps.json
sokulee/A050/A050_20160406_steps.json
sokulee/A051/A051_20160406_steps.json
sokulee/A052/A052_20160406_steps.json
sokulee/A053/A053_20160406_steps.json
sokulee/A054/A054_20160406_steps.json
sokulee/A055/A055_20160406_steps.json
sokulee/A056/A056_20160406_steps.json
sokulee/A057/A057_20160406_steps.json
sokulee/A058/A058_20160406_steps.json
sokulee/A059/A059_20160406_steps.json
sokulee/A060/A060_20160406_steps.json
sokulee/A061/A061_20160406_steps.json
sokulee/A062/A062_20160406_steps.json
sokulee/A063/A063_20160406_steps.json
sokulee/A064

sokulee/A090/A090_20160408_steps.json
sokulee/A091/A091_20160408_steps.json
sokulee/A092/A092_20160408_steps.json
sokulee/A093/A093_20160408_steps.json
sokulee/A094/A094_20160408_steps.json
sokulee/A095/A095_20160408_steps.json
sokulee/A096/A096_20160408_steps.json
sokulee/A097/A097_20160408_steps.json
sokulee/A01/A01_20160409_steps.json
sokulee/A02/A02_20160409_steps.json
sokulee/A03/A03_20160409_steps.json
sokulee/A04/A04_20160409_steps.json
sokulee/A05/A05_20160409_steps.json
sokulee/A06/A06_20160409_steps.json
sokulee/A07/A07_20160409_steps.json
sokulee/A08/A08_20160409_steps.json
sokulee/A09/A09_20160409_steps.json
sokulee/A010/A010_20160409_steps.json
sokulee/A011/A011_20160409_steps.json
sokulee/A012/A012_20160409_steps.json
sokulee/A013/A013_20160409_steps.json
sokulee/A014/A014_20160409_steps.json
sokulee/A015/A015_20160409_steps.json
sokulee/A016/A016_20160409_steps.json
sokulee/A017/A017_20160409_steps.json
sokulee/A018/A018_20160409_steps.json
sokulee/A019/A019_20160409_ste

sokulee/A035/A035_20160412_steps.json
sokulee/A036/A036_20160412_steps.json
sokulee/A037/A037_20160412_steps.json
sokulee/A038/A038_20160412_steps.json
sokulee/A039/A039_20160412_steps.json
sokulee/A040/A040_20160412_steps.json
sokulee/A041/A041_20160412_steps.json
sokulee/A042/A042_20160412_steps.json
sokulee/A043/A043_20160412_steps.json
sokulee/A044/A044_20160412_steps.json
sokulee/A045/A045_20160412_steps.json
sokulee/A046/A046_20160412_steps.json
sokulee/A047/A047_20160412_steps.json
sokulee/A048/A048_20160412_steps.json
sokulee/A049/A049_20160412_steps.json
sokulee/A050/A050_20160412_steps.json
sokulee/A051/A051_20160412_steps.json
sokulee/A052/A052_20160412_steps.json
sokulee/A053/A053_20160412_steps.json
sokulee/A054/A054_20160412_steps.json
sokulee/A055/A055_20160412_steps.json
sokulee/A056/A056_20160412_steps.json
sokulee/A057/A057_20160412_steps.json
sokulee/A058/A058_20160412_steps.json
sokulee/A059/A059_20160412_steps.json
sokulee/A060/A060_20160412_steps.json
sokulee/A061

...

sokulee/A089/A089_20160505_steps.json
sokulee/A090/A090_20160505_steps.json
sokulee/A091/A091_20160505_steps.json
sokulee/A092/A092_20160505_steps.json
sokulee/A093/A093_20160505_steps.json
sokulee/A094/A094_20160505_steps.json
sokulee/A095/A095_20160505_steps.json
sokulee/A096/A096_20160505_steps.json
sokulee/A097/A097_20160505_steps.json
sokulee/A01/A01_20160506_steps.json
sokulee/A02/A02_20160506_steps.json
sokulee/A03/A03_20160506_steps.json
sokulee/A04/A04_20160506_steps.json
sokulee/A05/A05_20160506_steps.json
sokulee/A06/A06_20160506_steps.json
sokulee/A07/A07_20160506_steps.json
sokulee/A08/A08_20160506_steps.json
sokulee/A09/A09_20160506_steps.json
sokulee/A010/A010_20160506_steps.json
sokulee/A011/A011_20160506_steps.json
sokulee/A012/A012_20160506_steps.json
sokulee/A013/A013_20160506_steps.json
sokulee/A014/A014_20160506_steps.json
sokulee/A015/A015_20160506_steps.json
sokulee/A016/A016_20160506_steps.json
sokulee/A017/A017_20160506_steps.json
sokulee/A018/A018_20160506_ste

sokulee/A020/A020_20160509_steps.json
sokulee/A021/A021_20160509_steps.json
sokulee/A022/A022_20160509_steps.json
sokulee/A023/A023_20160509_steps.json
sokulee/A024/A024_20160509_steps.json
sokulee/A025/A025_20160509_steps.json
sokulee/A026/A026_20160509_steps.json
sokulee/A027/A027_20160509_steps.json
sokulee/A028/A028_20160509_steps.json
sokulee/A029/A029_20160509_steps.json
sokulee/A030/A030_20160509_steps.json
sokulee/A031/A031_20160509_steps.json
sokulee/A032/A032_20160509_steps.json
sokulee/A033/A033_20160509_steps.json
sokulee/A034/A034_20160509_steps.json
sokulee/A035/A035_20160509_steps.json
sokulee/A036/A036_20160509_steps.json
sokulee/A037/A037_20160509_steps.json
sokulee/A038/A038_20160509_steps.json
sokulee/A039/A039_20160509_steps.json
sokulee/A040/A040_20160509_steps.json
sokulee/A041/A041_20160509_steps.json
sokulee/A042/A042_20160509_steps.json
sokulee/A043/A043_20160509_steps.json
sokulee/A044/A044_20160509_steps.json
sokulee/A045/A045_20160509_steps.json
sokulee/A046

sokulee/A031/A031_20160512_steps.json
sokulee/A032/A032_20160512_steps.json
sokulee/A033/A033_20160512_steps.json
sokulee/A034/A034_20160512_steps.json
sokulee/A035/A035_20160512_steps.json
sokulee/A036/A036_20160512_steps.json
sokulee/A037/A037_20160512_steps.json
sokulee/A038/A038_20160512_steps.json
sokulee/A039/A039_20160512_steps.json
sokulee/A040/A040_20160512_steps.json
sokulee/A041/A041_20160512_steps.json
sokulee/A042/A042_20160512_steps.json
sokulee/A043/A043_20160512_steps.json
sokulee/A044/A044_20160512_steps.json
sokulee/A045/A045_20160512_steps.json
sokulee/A046/A046_20160512_steps.json
sokulee/A047/A047_20160512_steps.json
sokulee/A048/A048_20160512_steps.json
sokulee/A049/A049_20160512_steps.json
sokulee/A050/A050_20160512_steps.json
sokulee/A051/A051_20160512_steps.json
sokulee/A052/A052_20160512_steps.json
sokulee/A053/A053_20160512_steps.json
sokulee/A054/A054_20160512_steps.json
sokulee/A055/A055_20160512_steps.json
sokulee/A056/A056_20160512_steps.json
sokulee/A057

sokulee/A047/A047_20160515_steps.json
sokulee/A048/A048_20160515_steps.json
sokulee/A049/A049_20160515_steps.json
sokulee/A050/A050_20160515_steps.json
sokulee/A051/A051_20160515_steps.json
sokulee/A052/A052_20160515_steps.json
sokulee/A053/A053_20160515_steps.json
sokulee/A054/A054_20160515_steps.json
sokulee/A055/A055_20160515_steps.json
sokulee/A056/A056_20160515_steps.json
sokulee/A057/A057_20160515_steps.json
sokulee/A058/A058_20160515_steps.json
sokulee/A059/A059_20160515_steps.json
sokulee/A060/A060_20160515_steps.json
sokulee/A061/A061_20160515_steps.json
sokulee/A062/A062_20160515_steps.json
sokulee/A063/A063_20160515_steps.json
sokulee/A064/A064_20160515_steps.json
sokulee/A065/A065_20160515_steps.json
sokulee/A066/A066_20160515_steps.json
sokulee/A067/A067_20160515_steps.json
sokulee/A068/A068_20160515_steps.json
sokulee/A069/A069_20160515_steps.json
sokulee/A070/A070_20160515_steps.json
sokulee/A071/A071_20160515_steps.json
sokulee/A072/A072_20160515_steps.json
sokulee/A073

sokulee/A063/A063_20160518_steps.json
sokulee/A064/A064_20160518_steps.json
sokulee/A065/A065_20160518_steps.json
sokulee/A066/A066_20160518_steps.json
sokulee/A067/A067_20160518_steps.json
sokulee/A068/A068_20160518_steps.json
sokulee/A069/A069_20160518_steps.json
sokulee/A070/A070_20160518_steps.json
sokulee/A071/A071_20160518_steps.json
sokulee/A072/A072_20160518_steps.json
sokulee/A073/A073_20160518_steps.json
sokulee/A074/A074_20160518_steps.json
sokulee/A075/A075_20160518_steps.json
sokulee/A076/A076_20160518_steps.json
sokulee/A077/A077_20160518_steps.json
sokulee/A078/A078_20160518_steps.json
sokulee/A079/A079_20160518_steps.json
sokulee/A080/A080_20160518_steps.json
sokulee/A081/A081_20160518_steps.json
sokulee/A082/A082_20160518_steps.json
sokulee/A083/A083_20160518_steps.json
sokulee/A084/A084_20160518_steps.json
sokulee/A085/A085_20160518_steps.json
sokulee/A086/A086_20160518_steps.json
sokulee/A087/A087_20160518_steps.json
sokulee/A088/A088_20160518_steps.json
sokulee/A089

In [13]:
total_step

[132951,
 101387,
 76686,
 53452,
 42746,
 15660,
 14163,
 24262,
 132124,
 210512,
 281407,
 299711,
 414384,
 348594,
 287331,
 376051,
 313656,
 360350,
 412639,
 348440,
 249294,
 388552,
 292799,
 234235,
 166337,
 114013,
 104646,
 59966,
 35489,
 10380,
 13684,
 28165,
 132417,
 307230,
 199069,
 306947,
 444218,
 258738,
 330182,
 270617,
 310441,
 387549,
 447367,
 280255,
 256107,
 278044,
 250791,
 185034,
 134901,
 135230,
 94332,
 62740,
 49809,
 20090,
 19409,
 45704,
 118733,
 228304,
 168717,
 281106,
 397473,
 244670,
 329784,
 255933,
 260202,
 359158,
 416158,
 245530,
 236925,
 393144,
 217196,
 187424,
 174243,
 144479,
 83613,
 80491,
 45205,
 52413,
 26801,
 40639,
 140689,
 208913,
 244173,
 230388,
 358278,
 237569,
 331817,
 208112,
 267062,
 402084,
 380385,
 320028,
 310745,
 250177,
 259987,
 206089,
 170789,
 177701,
 111844,
 75562,
 41586,
 18016,
 15553,
 32679,
 178663,
 177343,
 214976,
 259437,
 322847,
 359254,
 379264,
 359041,
 332091,
 384626,
 4

In [14]:
weekday = []
hour = []
for i in range(7):
    for j in range(24):
        weekday.append(i)
        hour.append(j)  

In [15]:
data = pd.DataFrame({'weekday' : weekday ,'hour' : hour, 'step' : total_step})

In [16]:
data

,hour,step,weekday
0,0,132951,0
1,1,101387,0
2,2,76686,0
3,3,53452,0
4,4,42746,0
5,5,15660,0
6,6,14163,0
7,7,24262,0
8,8,132124,0
9,9,210512,0


In [17]:
filename = 'heatmap_day.csv'
data.to_csv(filename, index=False, encoding='utf-8')